## <5-2 문장 쌍 분류 모델 학습하기>  

문장 쌍 분류 모델의 데이터 전처리와 학습 과정을 실습으로 진행해 보겠습니다.

### 전제와 가설을 검증하는 자연어 추론 모델 만들기
---
<**1단계**> **코랩 노트북 초기화하기**   
이번 실습은 웹 브라우저에서 다음 주소(bit.ly/3ajrGuo)에 접속하면 코랩 환경에서 수행할 수 있습니다. 이전 실습과 마찬가지로 코랩에서 `[내 드라이브에 복사]`와 런터임 유형에서 하드웨어 가속을 `[GPU]`로 선택합니다. 

<**2단계**> **각종 설정하기**  
1단계 코랩 노트북 초기화 과정에서 하드웨어 가속기로 TPU를 선택했다면 다음 코드를 실행하고, GPU를 선택했다면 실행하지 않습니다.  

In [12]:
#코드 5-1 TPU 관련 패키지 설치
#!pip install cloud-tpu-client==0.10 http://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

다음 코드를 실행해 TPU 이외에 의존성 있는 패키지를 설치하고 코드 5-3도 실행해 자신의 구글 드라이브를 코랩 노트북과 연결합니다.

In [13]:
#코드 5-2 의존성 패키지 설치
!pip install ratsnlp

In [14]:
#코드 5-3 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이번 실습에서는 `kcbert-base` 모델을 인공지능 기업 업스테이지가 공개한 `KLUE-NLI` 데이터로 파인튜닝해 볼 계획입니다. 다음 코드를 실행하면 관련 설정을 할 수 있습니다. 우리는 문장 쌍 분류 모델을 학습할 예정이므로 `downstream_task_name`에 `pair-classification`이라고 적어 둡니다. 이외에 이 코드에서 설정하는 각종 인자에 관한 설명은 <4-2>절을 참고바랍니다.


In [15]:
#코드 5-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_task_name="pair-classification",
    downstream_corpus_name="klue-nli",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=5,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

다음 코드는 랜덤 시드를 설정합니다. 이 코드는 `args`에 지정된 시드로 고정하는 역할을 합니다.  
그리고 코드 5-6을 실행해 각종 로그를 출력하는 로거를 설정합니다.

In [16]:
#코드 5-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [17]:
#코드 5-6 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_siz

<**3단계**> **말뭉치 내려받기**  

다음 코드를 실행하면 KLUE-NLI 데이터를 내려받습니다. `corpus_name`에 해당하는 말뭉치 `(klue-nli)`를 `downstream_corpus_root_dir` 아래(/root/Korpora)에 저장해 둡니다.

In [18]:
#코드 5-7 말뭉치 내려받기
nlpbook.download_downstream_dataset(args)

INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!


<**4단계**> **토크나이저 준비하기**  

다음 코드를 실행해 `pretrained_model_name`에 해당하는 모델(`kcbert-base`)이 사용하는 토크나이저를 선언합니다.

In [19]:
#코드 5-8 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

<**5단계**> **데이터 전처리하기**   

다음 코드를 수행하면 학습 데이터셋을 만들 수 있습니다. `KlueNLICorpus` 클래스는 JSON 파일 형식의 KLUE-NLI 데이터를 문장(전제 + 가설)과 레이블(참, 거짓, 중립)로 읽어들입니다. `KlueNLICorpus`는 `ClassificationDataset`이 요구하면 이 문장과 레이블을 `ClassificationDataset`에 제공합니다.

In [20]:
#코드 5-9 학습 데이터셋 구축
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset
corpus = KlueNLICorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_train_BertTokenizer_64_klue-nli_pair-classification [took 1.519 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_train_BertTokenizer_64_klue-nli_pair-classification [took 1.519 s]


`ClassificationDataset` 클래스는 `KlueNLICorpus`와 코드 5-8에서 선언해 둔 토크나이저를 품고 있습니다. `ClassificationDataset`는 `KlueNLICorpus`가 넘겨준 문장(전제, 가설)과 레이블 각각을 모델이 학습할 수 있는 형태(`ClassificationFeatures`)로 가공합니다. 다시 말해 전제와 가설 2개 문장을 각각 토큰화하고 이를 인덱스로 변환하는 한편, 레이블 역시 정수로 바꿔주는 역할을 합니다(`entailment: 0, contradiction: 1, neutral: 2`).  

cf) ClassificationFeatures는 데이터 인스턴스를 input_ids, attention_mask, token_type_ids, label 등 4가지로 변환한 자료입니다.

그러면 `KlueNLICorpus`가 넘겨준 0번 데이터를 봅시다.  

- **전제:** 100분간 잘껄 그래도 소닉붐땜에 2점준다  
- **가설:** 100분간 잤다.
- **레이블:** contradiction 

그러면 `ClassificationDataset`은 우선 전제와 가설을 각각 토큰화한 뒤 '`[CLS] 전제 [SEP] 가설 [SEP]`' 형태로 이어 붙입니다. 토큰화 결과를 확인해 보면 다음과 같습니다. 시퀀스 뒷부분에 `[PAD]` 토큰이 많이 붙어 있는데요, 이는 전제와 가설을 모두 합친 토큰 시퀀스의 길이가 코드 5-4에서 정한 `max_seq_length`인 64보다 짧아서입니다. 이보다 길면 64로 줄입니다.

In [21]:
ClassificationDataset

ratsnlp.nlpbook.classification.corpus.ClassificationDataset

코랩에서 코드 5-9를 실행한 뒤 `train_dataset[0]`을 입력하면 다음과 같은 결과를 확인할 수 있습니다. `KlueNLICorpus`가 넘겨준 0번 데이터가 최종적으로 `ClassificationDataset`의 0번 인스턴스로 변환된 것입니다. 이처럼 `ClassificationDataset`이 가지고 있는 모든 인스턴스는 인덱스로 접근할 수 있습니다.

In [22]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=1)

`train_dataset[0].input_ids`는 `'[CLS] 전제 [SEP] 가설 [SEP]'`를 토큰화한 뒤 인덱싱한 결과입니다.  
`train_dataset[0].attention_mask`는 해당 위치의 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타냅니다.

`train_dataset[0].token_type_ids`는 ***세그먼트(segment)*** 정보입니다. `'[CLS] 전제 [SEP]'`에 해당하는 첫 번째 세그먼트는 0, `가설 [SEP]`에 해당하는 두 번째 세그먼트는 1, 나머지 패딩에 속하는 세 번째 세그먼트는 0을 줍니다. 전제와 가설의 토큰 수는 각각 12, 5개이므로 첫 번째 세그먼트(0으로 채움)의 길이는 `[CLS]`와 `[SEP]`를 합쳐 14, 두 번째 세그먼트(1로 채움)는 `[SEP]`를 포함해 6이 됩니다. 마지막 세그먼트(0으로 채움)의 길이는 64(`max_seq_length`) - 14(첫 번째 세그먼트 길이) - 6(두 번째 세그먼트 길이), 즉 44가 됩니다.

한편 `KlueNLICorpus`와 `ClassificationDataset`의 역할과 자세한 구현 내용은 다음 링크를 참고하세요.  
KLUE-NLI 말고 내가 가진 말뭉치를 문장 쌍 분류 모델 학습용 데이터로 전처리 하는 방법 역시 안내하고 있습니다.

- ratsgo.github.io/nlpbook/docs/pair_cls/detail

다음 코드를 실행하면 학습할 때 쓰이는 데이터 로더를 만들 수 있습니다. 학습용 데이터 로더는 `ClassificationDataset` 클래스가 들고 있는 전체 인스턴스 가운데 배치 크기(코드 5-4에서 정의한 `args`의 `batch_size`)만큼의 인스턴스들을 비복원(`replacement=False`) 랜덤 추출(`RandomSampler`)한 뒤 이를 배치 형태로 가공(`nlpbook.data_collator`)해 모델에 공급하는 역할을 합니다.

In [23]:
#코드 5-10 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

다음 코드를 실행하면 평가용 데이터 로더를 구축할 수 있습니다.  
평가용 데이터 로더는 배치 크기(코드 5-4에서 정의한 `args`의 `batch_size`)만큼의 인스턴스를 순서대로 추출(`Sequential Sampler`)한 후 이를 배치 형태로 가공(`nlpbook.data_collator`)해 모델에 공급합니다. 

In [35]:
#코드 5-11 평가용 데이터 로더 구축
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_test_BertTokenizer_64_klue-nli_pair-classification [took 0.391 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_test_BertTokenizer_64_klue-nli_pair-classification [took 0.391 s]


NameError: ignored

<**6단계**> **모델 불러오기**  
다음 코드를 수행해 모델을 초기화합니다. 프리트레인을 마친 BERT로 `kcbert-base`를 사용합니다. 코드 5-4에서 `pretrained_model_name`을 `beomi/kcbert-base`로 지정했기 떄문입니다.  
물론 허깅페이스 모델 허브에 등록된 모델이라면 다른 모델 역시 사용할 수 있습니다.

`BertForSequenceClassification`은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈을 덧붙은 형태의 모델 클래스입니다. 이 클래스는 4-2절 문서 분류에 썼던 것과 동일합니다.

In [27]:
#코드 5-12 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

<**7단계**> **모델 학습시키기**  
다음 코드를 실행하면 문장 쌍 분류용 태스크를 정의할 수 있습니다. 모델은 코드 5-12에서 준비한 모델 클래스를 `ClassificationTask`에 포함합니다. `ClassificationTask` 클래스에는 옵티마이저, 러닝 레이트 스케줄러가 정의돼 있는데요, 옵티마이저로는 아담(Adam), 러닝 레이트 스케줄러로는 `ExponentialLR`을 사용합니다.

In [28]:
#코드 5-13 태스크 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

cf) `ClassificationTask`의 자세한 구현 내용은 다음 링크를 참고하시면 좋을 것 같습니다!  

- ratsgo.github.io/nlpbook/docs/pair_cls/detail

다음 코드를 실행하면 트레이너를 정의할 수 있습니다. 이 트레이너는 파이토치 라이트닝 라이브러리의 도움을 받아 GPU/TPU 설정, 로그 및 체크포인트 등 귀찮은 설정들을 알아서 해줍니다.

In [29]:
#코드 5-14 트레이너 정의
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


다음 코드처럼 트레이너의 `fit()` 함수를 호출하면 학습을 시작합니다.

In [32]:
# 코드 5-15 학습 개시
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

NameError: ignored

지금까지 우리는 문장 쌍 분류의 대표 과제인 자연어 추론 모델 학습을 실습해 보았습니다.  
문장 쌍 분류는 <4-2>절 문서 분류 과제와 태스크 모듈 구조 등에서 본질적으로 다르지 않습니다. 입력 문서가 1개냐(문서 분류) 2개냐(문장 쌍 분류)의 차이가 있을 뿐입니다.  
이로써 문서 분류 모델은 한 문서의 범주, 문장 쌍 분류 모델은 두 문서 사이의 관계를 분류할 수 있게 됩니다.